# Convert a Kaggle Dataset into a Postgres Database

In this notebook, I go over the Goodreads-book Dataset on Kaggle and store the data in a PostgresSQL Database. I go over the following:
1. Loading the dataset
2. Performing preliminary analysis to understand the dataset
3. Map out a Entity-Relationship Model to store the data in a database
4. Create database tables
5. Populate the database tables from the dataset using Python
6. Perform complex SQL queries to extract insights from the data

In [11]:
import pandas as pd
import numpy as np

In [12]:
!ls

Homework 1.ipynb books.csv        db_cred.json


## Import Data and Data Exploration

In [13]:
#import the data:
df = pd.read_csv('books.csv')

In [15]:
df.head(3)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1.0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,439785960,9.780000e+12,eng,652.0,2095690.0,27591.0,9/16/2006,Scholastic Inc.
1,2.0,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,439358078,9.780000e+12,eng,870.0,2153167.0,29221.0,9/1/2004,Scholastic Inc.
2,4.0,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9.780000e+12,eng,352.0,6333.0,244.0,11/1/2003,Scholastic


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  float64
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  float64
 6   language_code       11123 non-null  object 
 7     num_pages         11123 non-null  float64
 8   ratings_count       11123 non-null  float64
 9   text_reviews_count  11123 non-null  float64
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(6), object(6)
memory usage: 1.0+ MB


In [17]:
df.shape

(11127, 12)

In [18]:
for col in df.columns:
    print(f"{col}: ",len(df[col].unique()))

bookID:  11124
title:  10349
authors:  6640
average_rating:  210
isbn:  11124
isbn13:  24
language_code:  28
  num_pages:  998
ratings_count:  5295
text_reviews_count:  1823
publication_date:  3679
publisher:  2291


There's one thing we can clearly observe here, that all three columns: bookId, isbn and isbn13 have about 11123 records which is the length of the dataset. Isbn numbers can be used to identify a book uniquely and the dataset confirms this.

Furthermore, it is important for us to acknowledge that there are multiple authors that wrote a book. For instance, Expelled from Eden: A William T. Vollmann Reade was written by William T. Vollmann,Larry McCaffery,Michael and so on. 

We also notice that the title is less than 11123, which begs the question how there are not the same number of unique titles as there are bookIds.

In [19]:
df[df['title'].duplicated()]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
10,16.0,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Douglas Adams/Stephen Fry,4.22,739322206,9.780000e+12,eng,6.0,1266.0,253.0,3/23/2005,Random House Audio
24,35.0,The Lord of the Rings (The Lord of the Rings ...,J.R.R. Tolkien/Alan Lee,4.50,618260587,9.780000e+12,en-US,1216.0,1618.0,140.0,10/1/2002,Houghton Mifflin Harcourt
37,63.0,The Changeling,Kate Horsley,3.55,1590301943,9.780000e+12,eng,339.0,301.0,43.0,4/12/2005,Shambhala
40,68.0,The Known World,Edward P. Jones/Kevin R. Free,3.83,006076273X,9.780000e+12,en-US,14.0,55.0,12.0,6/15/2004,HarperAudio
41,69.0,The Known World,Edward P. Jones,3.83,60749911,9.780000e+12,eng,576.0,22.0,3.0,6/15/2004,Harper
...,...,...,...,...,...,...,...,...,...,...,...,...
11079,45463.0,Encuentro en el Ártico (Artemis Fowl #2),Eoin Colfer/Ana Alcaina Pérez,3.95,307343103,9.780000e+12,spa,320.0,15.0,3.0,7/5/2005,Montena
11080,45465.0,Artemis Fowl (Artemis Fowl #1),Eoin Colfer/Nathaniel Parker,3.84,1400085918,9.780000e+12,eng,6.0,55.0,14.0,4/6/2004,Listening Library
11082,45479.0,The Subtle Knife (His Dark Materials #2),Philip Pullman/Ian Beck,4.13,375846727,9.780000e+12,eng,326.0,677.0,77.0,9/1/2007,Alfred A. Knopf Books for Young Readers
11084,45485.0,The Subtle Knife (His Dark Materials #2),Philip Pullman,4.13,807210471,9.780000e+12,eng,8.0,39.0,8.0,1/28/2000,Listening Library


In [20]:
df[df['title'] == 'The Known World']

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
39,67.0,The Known World,Edward P. Jones,3.83,61159174,9.780000e+12,eng,388.0,29686.0,2626.0,8/29/2006,Amistad
40,68.0,The Known World,Edward P. Jones/Kevin R. Free,3.83,006076273X,9.780000e+12,en-US,14.0,55.0,12.0,6/15/2004,HarperAudio
41,69.0,The Known World,Edward P. Jones,3.83,60749911,9.780000e+12,eng,576.0,22.0,3.0,6/15/2004,Harper
3193,11684.0,The Known World,Edward P. Jones,3.83,7195303,9.780000e+12,eng,388.0,449.0,83.0,10/1/2004,Harper Perennial


In [21]:
df[df['title'] == 'The Changeling']

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
36,61.0,The Changeling,Zilpha Keatley Snyder,4.17,595321801,9.780000e+12,eng,228.0,1176.0,96.0,6/8/2004,iUniverse
37,63.0,The Changeling,Kate Horsley,3.55,1590301943,9.780000e+12,eng,339.0,301.0,43.0,4/12/2005,Shambhala


These examples show that there are about 778 records of similar title but belonging to different publishers, different publication date or different language code. It can also be seen that the ISBN13 is almost similar for these records, with only the ISBN number being unique across books. While we could use the ISBN number to uniquely identify a book, it seems like the ISBN number is registered as a string, not a number. Therefore, we can rely on the bookId being the unique identifier of a book. 



## Entity Relation Modeling & Database Normalization

In [22]:
df.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

In order to understand how we can craete an ER model, we can start by examining how each column is dependant on the bookId

BooksData('bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13','language_code', '  num_pages', 'ratings_count', 'text_reviews_count','publication_date', 'publisher')

We can safely state that columns 'title', 'authors', 'average_rating', 'isbn', 'isbn13','language_code', ' num_pages', 'ratings_count', 'text_reviews_count','publication_date', 'publisher' are dependant on bookID as each of these columns uniquely identify with a book. 

However, since there are multiple authors in the authors column we can conclude there is a many to many relationship between books and authors.
- A book can be written by multiple authors
- An author can write multiple books

Therefore, we can split the book's entity and create a author entity: \
**Books**('bookID', 'title', 'average_rating', 'isbn', 'isbn13','language_code', '  num_pages', 'ratings_count', 'text_reviews_count','publication_date', 'publisher') \
**Author**( AuthorID, AuthorName) \
And then we can create a BooksAuthor Table to capture the Books-Author relationship as follows: \
**BooksAuthor**( AuthorID, BookID)

In the Books Table, the publisher column is a string, and there were 2291 unique publishers. Assuming that publishers are the one's who publish a book, we can say that the relationship between Books and Publisher is a One to Many Relationship which can be captured as follows:
- One book can have only one publisher
     - This assumption is captured from the data wherein we saw that for different titles there were different publisher, even though they could just be one publisher.
     - For instance, HarperAudio, Harper and Harper Perennial could be the same publisher but for the sake of simplicity, we'll assume they are different entities based on the fact that 3 different books(different bookID) were published by them despite having the same title:The Known World
- Publisher can publish 1 or many books

Therefore, we can create a Publisher Entity/Table to store a publisherID and the name of the publisher, and use the publisherID to reference the Books table, as follows: \
**Books**('bookID', 'title', 'average_rating', 'isbn', 'isbn13','language_code', '  num_pages', 'ratings_count', 'text_reviews_count','publication_date', 'PublisherId') \
**Publisher**( PublisherId, PublisherName)

This has the benefit of preserving referential integrity. For instance if a book published a publisher that only published that particular book in the entire dataset gets deleted, we can still have the information on the publisher

Furthermore, we can also create a new Table for the language code since there are only 27-28 languages in the dataset. This gets repeated in the database multiple times and we can save some space in the database by not repeating these strings for 11124 times. We can create a new table Language and use the languageID in that table to reference a languageID, instead of language_code, in the books table: \
**Language**( LanguageId, Language) \
**Books**( bookID, title, average_rating, isbn, isbn13, LanguageId, num_pages, ratings_count, text_reviews_count, publication_date, PublisherId)

These adjustments lead us to our final database design.

**Books**( bookID, title, average_rating, isbn, isbn13, LanguageId, num_pages, ratings_count, text_reviews_count, publication_date, PublisherId) \
**Author**( AuthorID, AuthorName) \
**BooksAuthor**( AuthorID, BookID) \
**Publisher**( PublisherId, PublisherName) \
**Language**( LanguageId, Language) 



## Database Connection/ Creation

Before we proceed, I noticed that there were 4 rows of data with null value which could break our data import. In order to deal with this, we'll completely remove these rows for this project

In [23]:
df[df['authors'].isnull()]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
3348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df.dropna(inplace=True)

In [25]:
df[df['authors'].isnull()]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher


In [1]:
import psycopg2
import json

In [2]:
with open('db_cred.json','r') as j:
    cred = json.load(j)

In [3]:
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host=cred["host"],
    port=cred['port'],
    database=cred["database"],
    user=cred["user"],
    password=cred["password"])

Connecting to the PostgreSQL database...


In [4]:
# create a cursor
cur = conn.cursor()
# execute a statement
print('PostgreSQL database version:')
cur.execute('SELECT version()')
# display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)

PostgreSQL database version:
('PostgreSQL 15.1 on aarch64-apple-darwin21.6.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.102), 64-bit',)


### Create Authors, Publishers and Language Table

In [30]:
createCmdAuthor = """ CREATE TABLE IF NOT EXISTS authors (
                author_id SERIAL PRIMARY KEY,
                author_name VARCHAR(255) NOT NULL
                )
            """
createCmdPublisher = """ CREATE TABLE IF NOT EXISTS publishers (
                publisher_id SERIAL PRIMARY KEY,
                publisher_name VARCHAR(255) NOT NULL
                )
            """
createCmdLanguage = """ CREATE TABLE IF NOT EXISTS language (
                language_id SERIAL PRIMARY KEY,
                language_code VARCHAR(30) NOT NULL
                )
            """

cur.execute(createCmdAuthor)
cur.execute(createCmdPublisher)
cur.execute(createCmdLanguage)
conn.commit()

### Populate the Authors Table

In [31]:
authorslist = df['authors'].unique()
len(authorslist)

6639

In [32]:
authorslist_final = set()
for author in authorslist:
        auth = author.split("/")
        for a in auth:
            authorslist_final.add(a)

In [33]:
len(authorslist_final)

9231

In [34]:
# create a cursor
cur = conn.cursor()

In [35]:
errorsAuthors = []
for i, authorName in enumerate(authorslist_final):
    #print(i+1, authorName)
    try:
        cur.execute(f"INSERT into authors VALUES({i+1}, '{authorName}')")
        conn.commit()
    except:
        errorsAuthors.append((i+1,authorName))
        #need to re-create connection
        conn = psycopg2.connect(
            host=cred["host"],
            port=cred['port'],
            database=cred["database"],
            user=cred["user"],
            password=cred["password"])
        cur = conn.cursor()
        continue


In [36]:
len(errorsAuthors)

81

In [37]:
errorsAuthors[:5]

[(38, "Fitz-James O'Brien"),
 (50, "Michael D'Orso"),
 (176, "Barron's"),
 (598, "Robert C. O'Brien"),
 (621, "Kevin O'Malley")]

There were about 81 authors where we weren't able to insert due to the way the name is typed leading to parsing issues. For instance "INSERT into authors VALUES(16, 'Michael D'Orso')" throws an error because the code is being executed the quote ' after D is being interpreted as the end of the quote that started before M. In order to avoid this, and for the scope of this project, we can parse the string Michael D'Orso as Michael DOrso and consider it acceptable

In [38]:
def removeQuote(s):
    s = s.replace("'", "")     
    return s

s = "Michael D'Orso"
print(removeQuote(s))

Michael DOrso


In [39]:
errorsAuthors2 = []
for entry in errorsAuthors:
    #print(i+1, authorName)
    try:
        cur.execute(f"INSERT into authors VALUES({entry[0]}, '{ removeQuote(entry[1]) }')")
        conn.commit()
    except:
        errorsAuthors2.append((entry[0],entry[1]))
        #need to re-create connection
        conn = psycopg2.connect(
            host=cred["host"],
            port=cred['port'],
            database=cred["database"],
            user=cred["user"],
            password=cred["password"])
        cur = conn.cursor()
        continue
    

In [40]:
len(errorsAuthors2)

0

### Populate the Language Table

In [44]:
languages = df['language_code'].unique()

In [45]:
errorsLang = []
for i, langCode in enumerate(languages):
    #print(i+1, authorName)
    try:
        cur.execute(f"INSERT into language VALUES({i+1}, '{langCode}')")
        conn.commit()
    except:
        errorsLang.append((i+1,langCode))
        #need to re-create connection
        conn = psycopg2.connect(
            host=cred["host"],
            port=cred['port'],
            database=cred["database"],
            user=cred["user"],
            password=cred["password"])
        cur = conn.cursor()
        continue


In [46]:
len(errorsLang)

0

### Populate the Publishers Table

In [47]:
publishers = df['publisher'].unique()
publishers

array(['Scholastic Inc.', 'Scholastic', 'Nimble Books', ..., 'Suma',
       'Panamericana Editorial', 'Editorial Presença'], dtype=object)

In [48]:
errorsPublishers = []
for i, publisher in enumerate(publishers):
    #print(i+1, authorName)
    try:
        cur.execute(f"INSERT into publishers VALUES({i+1}, '{publisher}')")
        conn.commit()
    except:
        errorsPublishers.append((i+1,publisher))
        #need to re-create connection
        conn = psycopg2.connect(
            host=cred["host"],
            port=cred['port'],
            database=cred["database"],
            user=cred["user"],
            password=cred["password"])
        cur = conn.cursor()
        continue


In [49]:
len(errorsPublishers)

69

In [50]:
errorsPublishers[:10]

[(52, "O'Reilly Media"),
 (88, "Sterling Children's Books"),
 (112, "Everyman's Library"),
 (227, "Everyman's Library 234"),
 (285, "Collector's Library"),
 (306, "G.P. Putnam's Sons"),
 (371, "Simon & Schuster Children's Publishing"),
 (395, "Children's Classics"),
 (416, "St. Martin's Griffin"),
 (423, "Hodder Children's Books")]

This is a similar problem to the Author's name where we are facing a parsing issue. Therefore we can simply re-implement what we did for the Authors Table

In [51]:
errorsPublishers2 = []
for entry in errorsPublishers:
    #print(i+1, authorName)
    try:
        cur.execute(f"INSERT into publishers VALUES({entry[0]}, '{ removeQuote(entry[1]) }')")
        conn.commit()
    except:
        errorsPublishers2.append((entry[0],entry[1]))
        #need to re-create connection
        conn = psycopg2.connect(
            host=cred["host"],
            port=cred['port'],
            database=cred["database"],
            user=cred["user"],
            password=cred["password"])
        cur = conn.cursor()
        continue
    

In [52]:
len(errorsPublishers2)

0

### Creating the Books Table

Now that we created the Author's, Publishers and Language Table, we can now create a Books Table which can reference the other tables

**Books**( bookID, title, average_rating, isbn, isbn13, LanguageId, num_pages, ratings_count, text_reviews_count, publication_date, PublisherId)



In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11123 entries, 0 to 11126
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  float64
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  float64
 6   language_code       11123 non-null  object 
 7     num_pages         11123 non-null  float64
 8   ratings_count       11123 non-null  float64
 9   text_reviews_count  11123 non-null  float64
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(6), object(6)
memory usage: 1.1+ MB


Here we will perform a few data pre-processing:
- The first thing we want to do is convert the bookID column into an int since we will be using it as a primary key field
- Convert publication_data into a date time object
- Convert ratings count, num_pages and text_reviews_count to int
- Convert the ISBN's to strings

In [54]:
df['bookID'] = df['bookID'].astype(dtype=int)

In [55]:
df['publication_date'] = pd.to_datetime(df['publication_date'], format="%m/%d/%Y")

In [56]:
df['ratings_count'] = df['ratings_count'].astype(int)
df['  num_pages'] = df['  num_pages'].astype(int)
df['text_reviews_count'] = df['text_reviews_count'].astype(int)

In [57]:
df.rename(columns={"  num_pages": "num_pages"}, inplace=True)

In [58]:
df['isbn'] = df['isbn'].astype(str)
df['isbn13'] = df['isbn13'].astype(str)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11123 entries, 0 to 11126
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   bookID              11123 non-null  int64         
 1   title               11123 non-null  object        
 2   authors             11123 non-null  object        
 3   average_rating      11123 non-null  float64       
 4   isbn                11123 non-null  object        
 5   isbn13              11123 non-null  object        
 6   language_code       11123 non-null  object        
 7   num_pages           11123 non-null  int64         
 8   ratings_count       11123 non-null  int64         
 9   text_reviews_count  11123 non-null  int64         
 10  publication_date    11123 non-null  datetime64[ns]
 11  publisher           11123 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 1.1+ MB


In [60]:
df.tail(2)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
11125,45639,Poor People,William T. Vollmann,3.72,60878827,9780000000000.0,eng,434,769,139,2007-02-27,Ecco
11126,45641,Las aventuras de Tom Sawyer,Mark Twain,3.91,8497646983,9790000000000.0,spa,272,113,12,2006-05-28,Edimat Libros


In [61]:
createBooksCmd = """
    CREATE TABLE IF NOT EXISTS books(
        book_id INT PRIMARY KEY,
        title VARCHAR(255) NOT NULL,
        average_rating REAL NOT NULL,
        isbn VARCHAR(255) NOT NULL,
        isbn13 VARCHAR(255) NOT NULL,
        num_pages INT NOT NULL,
        ratings_count INT NOT NULL,
        text_reviews_count INT NOT NULL,
        publication_date TIMESTAMP,
        publisher_id INT NOT NULL,
        language_id INT NOT NULL,
        FOREIGN KEY (publisher_id) REFERENCES publishers (publisher_id),
        FOREIGN KEY (language_id) REFERENCES language (language_id)
    
    )

"""
cur.execute(createBooksCmd)
conn.commit()

### Populate the Books Table

Before we proceed to populate the Book's Table, we need to write a function that will return the foreign key id for publisher and language code

In [62]:
def getPublisherID(publisher_name):
    query = f"SELECT publisher_id FROM publishers WHERE publisher_name='{publisher_name}'"
    cur.execute(query)
    return cur.fetchone()[0]

def getLanguageID(language_code):
    query = f"SELECT language_id FROM language WHERE language_code='{language_code}'"
    cur.execute(query)
    return cur.fetchone()[0]

In [63]:
books_df = list(df.itertuples(index=False, name=None))

In [64]:
for item in books_df:
    query = f"""
        INSERT INTO books(
        book_id, title, average_rating, isbn, isbn13, language_id, num_pages, ratings_count, text_reviews_count, publication_date, publisher_id
        ) VALUES (
        {item[0]},'{removeQuote(item[1])}',{item[3]},'{item[4]}','{item[5]}',{getLanguageID(item[6])},
        {item[7]},{item[8]},{item[9]},'{item[10]}',{getPublisherID( removeQuote(item[11]) )}
        )
    """
    cur.execute(query)
conn.commit()

### Create and Populate the BooksAuthor Table

With the Books Table now in place along with the Authors Table, we now need to create a BooksAuthor Table that will capture the relationship between books and authors

Previously we decided that the BooksAuthor table will look like this: \
**BooksAuthor**( AuthorID, BookID)

In [65]:
createBooksAuthCmd = """
    CREATE TABLE IF NOT EXISTS books_author(
        book_id INT NOT NULL,
        author_id INT NOT NULL,
        PRIMARY KEY (book_id, author_id),
        FOREIGN KEY (book_id) references books (book_id),
        FOREIGN KEY (author_id) references authors (author_id)
    )
"""
cur.execute(createBooksAuthCmd)
conn.commit()

Since there are multiple authors for one book, we need to write a function that will do the following:

1. For every row of the books data
    1. Get the book_id
    2. Extract the author's section, and split it on "/". This will return an array
    3. For every author in the array:
        1. Run the removeQuote func
        2. Run a SQL query on the author's table to find the id
        3. Given the author_id, write a SQL insert using book_id,author_id

In [66]:
for row in books_df:
    book_id = row[0]
    authors = set(row[2].split("/")) #returns a set. See comment below
    for author_name in authors:
        author_name_fltr = removeQuote(author_name)
        find = f"SELECT author_id FROM authors WHERE author_name='{author_name_fltr}'"
        cur.execute(find)
        author_id = cur.fetchone()[0]
        insertCMD = f"""
            INSERT INTO books_author(book_id, author_id)
            VALUES ({book_id}, {author_id})"""
        cur.execute(insertCMD)
conn.commit()

"""
While running this i found that for book with id=2680, the author name is repeated twice:Xavier de C./Xavier de C./Joseph Rowe.
Therefore, we will introduce a set to take care of this.
"""

'\nWhile running this i found that for book with id=2680, the author name is repeated twice:Xavier de C./Xavier de C./Joseph Rowe.\nTherefore, we will introduce a set to take care of this.\n'

In [67]:
df[df['bookID']==2680]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
804,2680,Empire 2.0: A Modest Proposal for a United Sta...,Xavier de C./Xavier de C./Joseph Rowe,4.67,1556434952,9780000000000.0,eng,144,3,0,2004-05-04,North Atlantic Books


## Queries

Now that we have the database in place, we want to answer the following questions: 
1. What are the oldest 5 books in this dataset?
2. Which 5 language codes have the highest rating average?
3. Which 5 authors have the highest number of ratings?
4. Which publishers publish in the highest number of different languages?

In [60]:
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host=cred["host"],
    port=cred['port'],
    database=cred["database"],
    user=cred["user"],
    password=cred["password"])
cur = conn.cursor()

Connecting to the PostgreSQL database...


### What are the oldest 5 books in this dataset?

In [5]:
query = """
    SELECT title, publication_date FROM books
    ORDER BY publication_date ASC
    LIMIT 5
"""
cur.execute(query)
for row in cur:
    print(row[0])

Consider the Lilies
On Duties (De Officiis)
Agricola / Germania / Dialogue on Oratory
History of the Peloponnesian War: Bk. 1-2
History of the Peloponnesian War: Bk. 5-6


### Which 5 language codes have the highest rating average?

In [19]:
query = """
    SELECT books.language_id, language.language_code, AVG(books.average_rating) as avg
    FROM books
    INNER JOIN language
        ON books.language_id = language.language_id
    GROUP BY books.language_id,language.language_code
    ORDER BY avg DESC
    LIMIT 5
"""
cur.execute(query)
print(("book_id","language_code","average_rating"))
for row in cur:
    print(row)

('book_id', 'language_code', 'average_rating')
(22, 'wel', 5.0)
(26, 'gla', 4.46999979019165)
(14, 'zho', 4.456428595951626)
(25, 'tur', 4.420000076293945)
(27, 'ale', 4.360000133514404)


### Which 5 authors have the highest number of ratings?

**Note:**

Every book has a certain number of ratings_count. Every book has been written by one or more authors. 

In [58]:
query = """
    SELECT authors.author_name, SUM(books.ratings_count) AS no_of_ratings
    FROM books
    INNER JOIN books_author
        ON books.book_id = books_author.book_id
    INNER JOIN authors
        ON books_author.author_id = authors.author_id
    GROUP BY authors.author_name
    ORDER BY no_of_ratings DESC
    LIMIT 5
"""
cur.execute(query)
print(("auhthor", "# of ratings"))
for row in cur:
    print(row)

('auhthor', '# of ratings')
('J.K. Rowling', 9265287)
('Mary GrandPré', 8923980)
('J.R.R. Tolkien', 5622283)
('Stephenie Meyer', 4597666)
('Stephen King', 4288299)


### Which publishers publish in the highest number of different languages?

In [61]:
query = """
    SELECT publisher, COUNT(code) AS no_language_published
    FROM (SELECT publishers.publisher_name as publisher, language.language_code as code, COUNT(language.language_code) as lang_count
        FROM books
        INNER JOIN language
            ON books.language_id = language.language_id
        INNER JOIN publishers
            ON books.publisher_id = publishers.publisher_id
        GROUP BY publishers.publisher_name, language.language_code
        ORDER BY lang_count DESC) AS publisher_language_table
    GROUP BY publisher
    ORDER BY no_language_published DESC
    LIMIT 1
"""
cur.execute(query)
print(("publisher", "# of language published"))
for row in cur:
    print(row)

('publisher', '# of language published')
('Oxford University Press  USA', 6)


**Note**:
I want to reiterate a note I made earlier which is that authors and publishers that has names like O'Reilly Media,Sterling Children's Books appear as OReilly Media,Sterling Childrens in my database